In [1]:
################################################################################
#Michael Guerzhoy and Davi Frossard, 2016
#AlexNet implementation in TensorFlow, with weights
#Details: 
#http://www.cs.toronto.edu/~guerzhoy/tf_alexnet/
#
#With code from https://github.com/ethereon/caffe-tensorflow
#Model from  https://github.com/BVLC/caffe/tree/master/models/bvlc_alexnet
#Weights from Caffe converted using https://github.com/ethereon/caffe-tensorflow
#
#
################################################################################

from numpy import *
import os
#from pylab import *
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.cbook as cbook
import time
from scipy.misc import imread
from scipy.misc import imresize
import matplotlib.image as mpimg
from scipy.ndimage import filters
import urllib
from numpy import random
import skimage.io as io


import tensorflow as tf

from caffe_classes import class_names

train_x = zeros((1, 227,227,3)).astype(float32)
train_y = zeros((1, 1000))
xdim = train_x.shape[1:]
ydim = train_y.shape[1]



In [2]:
############################
#import database

import redisworks
client = redisworks.Root(host ='192.168.1.47',port=6379,db=0)
import MySQLdb as mysql
db=mysql.connect(host="alexadb.yishen.org",user="cs638",
                  passwd="Finalproject",db="cs638")
c = db.cursor()

In [38]:
from scipy.misc import imread
################################################################################
#Read Image, and change to BGR
images = []
nums = []
for i in range(10100,10200):
    #check wether the pic exist
    c.execute("""select picId from Pics where picId = %s """,(i,))
    res = c.fetchone()
    if(res is not None):
        res = res[0]
    else:
        continue

    #clear data
    result = None
    I = None
    im1 = None
    
    #get pic from redis
    #result = client['g'+str(i)]
    
    #get pic from local

    num = '%012d'%i
    #print(num)
    file_name = "COCO_train2014_"+ num +".jpg"
    #print(file_name)
    try:
        I = imread('D:/Machine Learning/fast-style-transfer-master/data/train2014\%s'%(file_name))
        if(I.ndim ==2):
            continue
        else:
            I = I[:,:,:3].astype(float32)
        #print (type(I[0][0]))
    except(IOError):
        print("here"+str(i))
        continue
    #name = 'g'+ num.lstrip('0')
    #print((I.ndim))
    #result1 = I.tolist()
    #result1 = list(result)
    
    #I = np.array(result1,dtype='float32')
    #if(I.ndim == 2):
        continue
    #resize to (227,227,3)
    import scipy.misc as resize
    im1 = resize.imresize(I,(227,227,3))
    im1.astype(float32)
    print (type(im1[0][0][0]))
    im1[:, :, 0], im1[:, :, 2] = im1[:, :, 2], im1[:, :, 0]
    
    images.append(im1)
    nums.append(i)


<class 'numpy.uint8'>
<class 'numpy.uint8'>
<class 'numpy.uint8'>
<class 'numpy.uint8'>
<class 'numpy.uint8'>
<class 'numpy.uint8'>
<class 'numpy.uint8'>
<class 'numpy.uint8'>
<class 'numpy.uint8'>
<class 'numpy.uint8'>


In [39]:


################################################################################

# (self.feed('data')
#         .conv(11, 11, 96, 4, 4, padding='VALID', name='conv1')
#         .lrn(2, 2e-05, 0.75, name='norm1')
#         .max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
#         .conv(5, 5, 256, 1, 1, group=2, name='conv2')
#         .lrn(2, 2e-05, 0.75, name='norm2')
#         .max_pool(3, 3, 2, 2, padding='VALID', name='pool2')
#         .conv(3, 3, 384, 1, 1, name='conv3')
#         .conv(3, 3, 384, 1, 1, group=2, name='conv4')
#         .conv(3, 3, 256, 1, 1, group=2, name='conv5')
#         .fc(4096, name='fc6')
#         .fc(4096, name='fc7')
#         .fc(1000, relu=False, name='fc8')
#         .softmax(name='prob'))

#In Python 3.5, change this to:
net_data = load(open("bvlc_alexnet.npy", "rb"), encoding="latin1").item()
#net_data = load("bvlc_alexnet.npy").item()

def conv(input, kernel, biases, k_h, k_w, c_o, s_h, s_w,  padding="VALID", group=1):
    '''From https://github.com/ethereon/caffe-tensorflow
    '''
    c_i = input.get_shape()[-1]
    assert c_i%group==0
    assert c_o%group==0
    convolve = lambda i, k: tf.nn.conv2d(i, k, [1, s_h, s_w, 1], padding=padding)
    
    
    if group==1:
        conv = convolve(input, kernel)
    else:
        input_groups =  tf.split(input, group, 3)   #tf.split(3, group, input)
        kernel_groups = tf.split(kernel, group, 3)  #tf.split(3, group, kernel) 
        output_groups = [convolve(i, k) for i,k in zip(input_groups, kernel_groups)]
        conv = tf.concat(output_groups, 3)          #tf.concat(3, output_groups)
    return  tf.reshape(tf.nn.bias_add(conv, biases), [-1]+conv.get_shape().as_list()[1:])



x = tf.placeholder(tf.float32, (None,) + xdim)


#conv1
#conv(11, 11, 96, 4, 4, padding='VALID', name='conv1')
k_h = 11; k_w = 11; c_o = 96; s_h = 4; s_w = 4
conv1W = tf.Variable(net_data["conv1"][0])
conv1b = tf.Variable(net_data["conv1"][1])
conv1_in = conv(x, conv1W, conv1b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=1)
conv1 = tf.nn.relu(conv1_in)

#lrn1
#lrn(2, 2e-05, 0.75, name='norm1')
radius = 2; alpha = 2e-05; beta = 0.75; bias = 1.0
lrn1 = tf.nn.local_response_normalization(conv1,
                                                  depth_radius=radius,
                                                  alpha=alpha,
                                                  beta=beta,
                                                  bias=bias)

#maxpool1
#max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool1 = tf.nn.max_pool(lrn1, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)


#conv2
#conv(5, 5, 256, 1, 1, group=2, name='conv2')
k_h = 5; k_w = 5; c_o = 256; s_h = 1; s_w = 1; group = 2
conv2W = tf.Variable(net_data["conv2"][0])
conv2b = tf.Variable(net_data["conv2"][1])
conv2_in = conv(maxpool1, conv2W, conv2b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv2 = tf.nn.relu(conv2_in)


#lrn2
#lrn(2, 2e-05, 0.75, name='norm2')
radius = 2; alpha = 2e-05; beta = 0.75; bias = 1.0
lrn2 = tf.nn.local_response_normalization(conv2,
                                                  depth_radius=radius,
                                                  alpha=alpha,
                                                  beta=beta,
                                                  bias=bias)

#maxpool2
#max_pool(3, 3, 2, 2, padding='VALID', name='pool2')                                                  
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool2 = tf.nn.max_pool(lrn2, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

#conv3
#conv(3, 3, 384, 1, 1, name='conv3')
k_h = 3; k_w = 3; c_o = 384; s_h = 1; s_w = 1; group = 1
conv3W = tf.Variable(net_data["conv3"][0])
conv3b = tf.Variable(net_data["conv3"][1])
conv3_in = conv(maxpool2, conv3W, conv3b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv3 = tf.nn.relu(conv3_in)

#conv4
#conv(3, 3, 384, 1, 1, group=2, name='conv4')
k_h = 3; k_w = 3; c_o = 384; s_h = 1; s_w = 1; group = 2
conv4W = tf.Variable(net_data["conv4"][0])
conv4b = tf.Variable(net_data["conv4"][1])
conv4_in = conv(conv3, conv4W, conv4b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv4 = tf.nn.relu(conv4_in)


#conv5
#conv(3, 3, 256, 1, 1, group=2, name='conv5')
k_h = 3; k_w = 3; c_o = 256; s_h = 1; s_w = 1; group = 2
conv5W = tf.Variable(net_data["conv5"][0])
conv5b = tf.Variable(net_data["conv5"][1])
conv5_in = conv(conv4, conv5W, conv5b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv5 = tf.nn.relu(conv5_in)

#maxpool5
#max_pool(3, 3, 2, 2, padding='VALID', name='pool5')
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool5 = tf.nn.max_pool(conv5, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

#fc6
#fc(4096, name='fc6')
fc6W = tf.Variable(net_data["fc6"][0])
fc6b = tf.Variable(net_data["fc6"][1])
fc6 = tf.nn.relu_layer(tf.reshape(maxpool5, [-1, int(prod(maxpool5.get_shape()[1:]))]), fc6W, fc6b)

#fc7
#fc(4096, name='fc7')
fc7W = tf.Variable(net_data["fc7"][0])
fc7b = tf.Variable(net_data["fc7"][1])
fc7 = tf.nn.relu_layer(fc6, fc7W, fc7b)

#fc8
#fc(1000, relu=False, name='fc8')
fc8W = tf.Variable(net_data["fc8"][0])
fc8b = tf.Variable(net_data["fc8"][1])
fc8 = tf.nn.xw_plus_b(fc7, fc8W, fc8b)



#prob
#softmax(name='prob'))
prob = tf.nn.softmax(fc8)

init = tf.global_variables_initializer()
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.run(init)

t = time.time()
# output = sess.run(prob, feed_dict = {x:[im1,im2]})
output = sess.run(fc8, feed_dict = {x:images})
################################################################################

#Output:

#print(output[0])
#print('here')
for input_im_ind in range(output.shape[0]):
    inds = argsort(output)[input_im_ind,:]
    print("Image", input_im_ind)
    for i in range(5):
        pass
        #print(class_names[inds[-1-i]], output[input_im_ind, inds[-1-i]])

print(time.time()-t)

Image 0
Image 1
Image 2
Image 3
Image 4
Image 5
Image 6
Image 7
Image 8
Image 9
0.20619702339172363


In [42]:
c = db.cursor()
for i in range(len(nums)):
    import json
    #print(output[0])
    s = json.dumps(output[i].tolist())
    #add the result to the sql database
    c.execute("""INSERT INTO CNNout (idCNNout,CNNoutcol)
         VALUES (%s, %s)""",(nums[i],s,))

In [43]:
db.commit()

In [44]:
d = db.cursor(mysql.cursors.DictCursor)

In [45]:
d.execute("""Select CNNoutcol from CNNout where idCNNout = %s""",(9,))

1

In [46]:
data = d.fetchone()

In [47]:
type(data)

dict

In [48]:
s = json.loads(data['CNNoutcol'])

In [49]:
type(s)

list

In [50]:
s[0]

-1.9006502628326416

In [51]:
c.close()

In [52]:
d.close()

In [53]:
nums[0]

10108

In [54]:
output[0]

array([ -2.48143220e+00,  -2.37729168e+00,   9.32337403e-01,
         4.08000052e-02,   1.06111392e-01,   1.81196702e+00,
        -6.67473555e-01,  -3.26984406e+00,  -3.85292816e+00,
        -2.08154154e+00,  -1.18211126e+00,  -3.50817180e+00,
        -2.00586963e+00,  -3.67008865e-01,   1.47871614e+00,
        -2.03247976e+00,  -3.12696314e+00,   3.92241478e-01,
        -1.22681713e+00,  -1.86699915e+00,  -3.69121170e+00,
        -7.54515648e-01,  -1.65500939e+00,  -8.81587744e-01,
         2.26927072e-01,  -1.11519742e+00,   5.19250035e-01,
        -2.12240219e+00,   3.25059354e-01,  -2.37590504e+00,
        -2.86131573e+00,  -2.40201139e+00,  -1.36893606e+00,
        -2.65294933e+00,  -2.84896970e+00,  -3.75455737e+00,
        -3.77389610e-01,  -4.62430811e+00,  -7.75163651e-01,
        -1.62834489e+00,  -2.23239541e+00,  -1.83454120e+00,
        -3.14033008e+00,  -1.49917293e+00,  -2.28077960e+00,
        -1.05861783e+00,  -1.04188788e+00,   3.13125283e-01,
        -4.70016003e+00,

In [55]:
type(images[0][0][0][0])

numpy.uint8

In [56]:
I = imread('D:/Machine Learning/fast-style-transfer-master/data/train2014\%s'%(file_name))

In [57]:
I.ndim

3